### Neural Matrix Factorization Recommender System

In [4]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score

# Veri setini yükleme
ratings_path = './ml-latest-small/ratings.csv'  # Kendi dosya yolunuza göre güncelleyin
df = pd.read_csv(ratings_path)

# Kullanıcı ve film ID'lerini yeniden kodlama (0'dan başlayarak)
user_ids = df['userId'].unique().tolist()
movie_ids = df['movieId'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

df['user'] = df['userId'].map(user2user_encoded)
df['movie'] = df['movieId'].map(movie2movie_encoded)

# Kullanıcı ve film sayıları
num_users = len(user2user_encoded)
num_movies = len(movie2movie_encoded)

# Eğitim ve test seti oluşturma
train, test = train_test_split(df, test_size=0.2, random_state=42)

# PyTorch Dataset sınıfı tanımlama
class MovieLensDataset(Dataset):
    def __init__(self, df):
        self.users = torch.tensor(df['user'].values, dtype=torch.long)
        self.movies = torch.tensor(df['movie'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['rating'].values, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.users[idx], self.movies[idx], self.ratings[idx]

train_dataset = MovieLensDataset(train)
test_dataset = MovieLensDataset(test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Neural Matrix Factorization Modeli
class NeuralMF(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(NeuralMF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, user, movie):
        user_embed = self.user_embedding(user)
        movie_embed = self.movie_embedding(movie)
        x = torch.cat([user_embed, movie_embed], dim=1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = NeuralMF(num_users, num_movies)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Model eğitimi
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for user, movie, rating in train_loader:
        optimizer.zero_grad()
        output = model(user, movie).squeeze()
        loss = criterion(output, rating)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# Tahmin yapma ve değerlendirme
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for user, movie, rating in test_loader:
        output = model(user, movie).squeeze()
        y_true.extend(rating.numpy())
        y_pred.extend(output.numpy())

# Binary sınıflandırma için değerlendirme metriklerini hesaplama
threshold = 3.5
y_true_binary = [1 if rating >= threshold else 0 for rating in y_true]
y_pred_binary = [1 if pred >= threshold else 0 for pred in y_pred]

auc = roc_auc_score(y_true_binary, y_pred_binary)
f1 = f1_score(y_true_binary, y_pred_binary)
accuracy = accuracy_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)

print(f'AUC: {auc:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')


/home/cevher/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1, Loss: 1.3867
Epoch 2, Loss: 0.9166
Epoch 3, Loss: 0.8484
Epoch 4, Loss: 0.7972
Epoch 5, Loss: 0.7595
Epoch 6, Loss: 0.7346
Epoch 7, Loss: 0.7119
Epoch 8, Loss: 0.6916
Epoch 9, Loss: 0.6745
Epoch 10, Loss: 0.6561
AUC: 0.6960
F1 Score: 0.7211
Accuracy: 0.6879
Recall: 0.6596
Precision: 0.7953
